# Script to conduct true and false positive rate analysis
___

In [14]:
#using Pkg
#Pkg.add(PackageSpec(name="KmerGMA", url = "https://github.com/Qile0317/KmerGMA.jl.git"))
using BenchmarkTools, BioSequences, BioAlignments, FASTX, Distances, Random #Main.KmerGMA, 
using Plots

module KmerGMA
    using BioSequences, FASTX, Distances, Random
     #WebBlast, Statistics, StringDistances, StatsBase, StatsPlots, DataFrames
    include("C:/Users/lu_41/.julia/dev/KmerGMA/src/simpleExplore.jl")
    include("C:/Users/lu_41/.julia/dev/KmerGMA/src/ExactMatch.jl")
    #include("kmerCount.jl")
    include("C:/Users/lu_41/.julia/dev/KmerGMA/src/GMAutils.jl")
    include("C:/Users/lu_41/.julia/dev/KmerGMA/src/RefGen.jl")
    include("C:/Users/lu_41/.julia/dev/KmerGMA/src/GMA.jl")
    include("C:/Users/lu_41/.julia/dev/KmerGMA/src/GMA_Nless.jl")
    include("C:/Users/lu_41/.julia/dev/KmerGMA/src/API.jl")
end

refile = "C:/Users/lu_41/.julia/dev/KmerGMA/test/Alp_V_ref.fasta"
single_ref_in_file = "C:/Users/lu_41/.julia/dev/KmerGMA/test/single_V_ref.fasta"
refile_cl2 = "C:/Users/lu_41/.julia/dev/KmerGMA/test/Alp_V_2.fasta"
human_refile = "C:/Users/lu_41/.julia/dev/KmerGMA/benchmark/Seqs/References/humans.fasta"

reference_KFV = KmerGMA.gen_ref(refile,6) # ignore for now
single_ref_KFV = KmerGMA.gen_ref(single_ref_in_file,6)
ref_cl2_KFV = KmerGMA.gen_ref(refile_cl2,6)

const NUCLEOTIDE_BITS = Dict{BioSequences.DNA, UInt}(
    DNA_A => unsigned(0),
    DNA_C => unsigned(1),
    DNA_G => unsigned(2),
    DNA_T => unsigned(3),
    DNA_N => unsigned(3) 
) 
println("complete")

complete


In [18]:
using BioSequences, FASTX, Random, BioAlignments#, KmerGMA
vgene = dna"caggtccagctggtgcagccaggggctgagctgaggaagcctggggctttgctgaaggtctcctgcaaggcttctggatacaccttcaccagctactacatagactgggtgcgacaggcccctggacaagggcttgggtgggtgggaagaattgaccctgaagacggtggcacaaattatgcacagaagttccagggcagagtcaccttgactgcagacacgtccaccagcacagcctacgtggagctgagcagtctgagatctgaggacacggccgtgtgttactgtgtgagaga"

296nt DNA Sequence:
CAGGTCCAGCTGGTGCAGCCAGGGGCTGAGCTGAGGAAG…GAGATCTGAGGACACGGCCGTGTGTTACTGTGTGAGAGA

Custom version of the KmerGMA algorithm for testing single sequences

In [16]:
const char_ints = Set(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

function cigar_to_UnitRange(aligned_obj)
    cigar_str::String = cigar(aligned_obj.aln.a.aln)
    curr_num, char_count, num_sum, lower = 0, 0, 0, 0
    n = length(cigar_str)
    for i in 1:n
        if i == n; return (lower+1):num_sum; end 
        if cigar_str[i] in char_ints
            curr_num = (curr_num * 10) + parse(Int, cigar_str[i])
        else
            char_count += 1
            if char_count == 1; lower = curr_num end
            num_sum += curr_num
            curr_num = 0
        end
    end
end

function gma_seq(;
    seq::LongSequence{DNAAlphabet{4}},
    k::Int64 = 6,
    refVec::Vector{Float64} = single_ref_KFV,
    windowsize::Int64 = 296,
    thr::Union{Int64, Float64} = 30,
    buff::Int64 = 50,
    curr_kmer_freq::Vector{Float64} = zeros(4096), 
    mask::UInt64 = unsigned(4095), 
    Nt_bits::Dict{DNA, UInt64} = NUCLEOTIDE_BITS,
    ScaleFactor::Float64 = 0.0833333333333333333333,
    score_model::AffineGapScoreModel{Int64} = AffineGapScoreModel(EDNAFULL, gap_open=-69, gap_extend=-1),
    vgene::LongSequence{DNAAlphabet{4}} = vgene,
    do_align::Bool = true,
    do_return_dists::Bool = false
    )

    resultVec = FASTA.Record[]
    if do_return_dists; dist_vec = Float64[] end 

    proceed::Bool, goal_ind::Int64, upcoming_n::Int64 = true, 0, 0

    #edge case
    sequence_length = length(seq)
    if sequence_length < windowsize; return end

    #initial operations for the first window  
    fill!(curr_kmer_freq, 0)
    KmerGMA.kmer_count!(str = view(seq, 1:windowsize), k = k,
    bins = curr_kmer_freq, mask = mask, Nt_bits = Nt_bits)
    currSqrEuc = Distances.sqeuclidean(refVec, curr_kmer_freq)

    #initializing variables
    CMI, stop, currminim = 2, true, currSqrEuc

    #left kmer initialization
    left_kmer = unsigned(0)
    for c in seq[1:k-1]
        left_kmer = (left_kmer << 2) + Nt_bits[c]
    end

    #right kmer initialization
    right_kmer = unsigned(0)
    for c in seq[windowsize-k+1:windowsize-1]
        right_kmer = (right_kmer << 2) + Nt_bits[c]
    end

    for i in 1:(sequence_length-windowsize)
        # first kmer
        left_kmer = ((left_kmer << 2) & mask) + Nt_bits[seq[i+k-1]]
        left_ind = -~left_kmer
        @views currSqrEuc -= (refVec[left_ind]-curr_kmer_freq[left_ind])^2 
        curr_kmer_freq[left_ind] -= 1
        @views currSqrEuc += (refVec[left_ind]-curr_kmer_freq[left_ind])^2

        # newest kmer
        right_kmer = ((right_kmer << 2) & mask) + Nt_bits[seq[i+windowsize-1]]
        right_ind = -~right_kmer
        @views currSqrEuc -= (refVec[right_ind]-curr_kmer_freq[right_ind])^2
        curr_kmer_freq[right_ind] += 1
        @views currSqrEuc += (refVec[right_ind]-curr_kmer_freq[right_ind])^2

        # convert to kmer Distance 
        kmerDist = currSqrEuc * ScaleFactor 
        if do_return_dists; push!(dist_vec, kmerDist) end 

        # minima finder
        if kmerDist < thr
            if kmerDist < currminim
                currminim = kmerDist
                CMI = i
                stop = false
            end
        elseif !stop
            if !proceed 
                if CMI > goal_ind
                    proceed = true
                end
            else # operations here aren't optimized but for actual long sequences the time is negligible
                goal_ind = CMI + windowsize - 1
                proceed = false 
                
                # alignment and matched unitrange
                left_ind, right_ind = max(CMI-buff,1), min(CMI+windowsize-1+buff,sequence_length)
                if do_align 
                    aligned_obj = pairalign(SemiGlobalAlignment(),vgene,view(seq,left_ind:right_ind),score_model)
                    aligned_UnitRange = cigar_to_UnitRange(cigar(aligned_obj.aln.a.aln))
                    seq_UnitRange = max(1, left_ind+first(aligned_UnitRange)-1):min(left_ind+last(aligned_UnitRange)-1, sequence_length)
                else
                    seq_UnitRange = left_ind:right_ind
                end

                #create record
                rec = FASTA.Record("test",view(seq, seq_UnitRange))
                push!(resultVec, rec)
                currminim = currSqrEuc
                stop = true
            end
        end
    end
    if do_return_dists; return resultVec, dist_vec end
    return resultVec
end

# it is expected to work until approximately 0.2 mutation rate with the selected threshold

gma_seq (generic function with 1 method)

Functions to introduce random substitutions and Indels to the reference sequence 

In [7]:
const mutation_dict = Dict{DNA, Vector{DNA}}(
    DNA_A => DNA[DNA_C, DNA_G, DNA_T],
    DNA_C => DNA[DNA_A, DNA_G, DNA_T],
    DNA_G => DNA[DNA_C, DNA_A, DNA_T],
    DNA_T => DNA[DNA_C, DNA_G, DNA_A]
)

const new_mutation_dict = Dict{DNA, Vector{LongSequence{DNAAlphabet{4}}}}(
    DNA_A => [dna"C", dna"G", dna"T"],
    DNA_C => [dna"A", dna"G", dna"T"],
    DNA_G => [dna"C", dna"A", dna"T"],
    DNA_T => [dna"C", dna"G", dna"A"]
)

function mutate_seq!(seq::LongSequence{DNAAlphabet{4}}, mut_rate::Float64)
    for i in 1:length(seq)
        if rand(1)[1] <= mut_rate
            seq[i] = rand(mutation_dict[seq[i]])
        end
    end
end

function indel_seq(seq::LongSequence{DNAAlphabet{4}}, indel_rate::Float64)
    curr_len, i = length(seq), 1
    while i < curr_len
        if rand(1)[1] <= indel_rate
            if rand(["", "del"]) == "del"
                seq = seq[1:i-1]*seq[i+1:end]*rand(new_mutation_dict[seq[i]])
                curr_len -= 1
            else
                seq = seq[1:i]*rand(new_mutation_dict[seq[i]])*seq[i+1:end]
                i += 2
            end
        else
            i += 1
        end
    end
    return seq
end 

function gma_tester(;mut_range::StepRangeLen = 0:0.005:0.2, n::Int = 50, seq::LongSequence{DNAAlphabet{4}} = vgene, do_mut::Bool = true)
    edit_dist_vec = Vector{Int64}[]
    for trial in 1:n
        curr_vec = Int64[]
        for curr_mut_rate in mut_range
            Random.seed!(trial)
            new_seq = copy(seq)
            if do_mut
                mutate_seq!(new_seq, curr_mut_rate)
            else
                new_seq = indel_seq(new_seq, curr_mut_rate)
            end

            search_seq = randdnaseq(400)*new_seq*randdnaseq(400)
            #lsq = length(search_seq)

            result = gma_seq(seq = search_seq, thr = 30.0, do_align = true, buff = 50, k = 4, refVec = KmerGMA.gen_ref(single_ref_in_file, 4), ScaleFactor = 0.125, curr_kmer_freq = zeros(4^4), mask = unsigned(4^4-1))
            if !isempty(result)
                result = KmerGMA.getSeq(result[1])
                push!(curr_vec, edit_dist(result, new_seq))
            else
                push!(curr_vec, 296)
            end
        end
        push!(edit_dist_vec, curr_vec)
    end
    return edit_dist_vec
end

gma_tester (generic function with 1 method)

Functions to measure the edit distance of two dna sequences

In [8]:
function common_prefix(s1::LongSequence{DNAAlphabet{4}}, s2::LongSequence{DNAAlphabet{4}})
    l = 0
    for (ch1, ch2) in zip(s1, s2)
        ch1 != ch2 && break
        l += 1
    end
    return l
end

function edit_dist(s1::LongSequence{DNAAlphabet{4}}, s2::LongSequence{DNAAlphabet{4}}; max_dist::Union{Integer, Nothing} = nothing)
    (s1 === missing) | (s2 === missing) && return missing
    len1, len2 = length(s1), length(s2)
    if len1 > len2
        s1, s2 = s2, s1
        len1, len2 = len2, len1
    end
    if max_dist !== nothing
        len2 - len1 > max_dist && return Int(max_dist + 1)
    end
    # prefix common to both strings can be ignored
    k = common_prefix(s1, s2)
    k == len1 && return len2 - k
    # first row of matrix set to distance between "" and s2[1:i]
    v = collect(1:(len2-k))
    current = 0
    for (i1, ch1) in enumerate(s1)
        i1 > k || continue
        left = current = i1 - k - 1
        if max_dist !== nothing
            value_lb = left - 1
        end
        for (i2, ch2) in enumerate(s2)
            i2 > k || continue
            above = current
            # cost on diagonal (substitution)
            current = left
            @inbounds left = v[i2 - k]
            if ch1 != ch2
                # minimum between substitution, deletion and insertion
                current = min(current + 1, above + 1, left + 1)
            end
            if max_dist !== nothing
                value_lb = min(value_lb, left)
            end
            @inbounds v[i2 - k] = current
        end
        if max_dist !== nothing
            value_lb > max_dist && return Int(max_dist + 1)
        end
    end
    if max_dist !== nothing
        current > max_dist && return Int(max_dist + 1 )
    end
    return current
end

edit_dist (generic function with 1 method)

Function to count true and false positives

In [9]:
# function for true positive rate
function count_mism(vecs::Vector{Vector{Int64}}, thr::Real = 10)
    res = [[0,0] for _ in 1:length(vecs[1])]

    for i in 1:length(vecs)
        for j in 1:length(vecs[i])
            if vecs[i][j] < thr 
                res[j][1] += 1
            else
                res[j][2] += 1
            end
        end
    end

    # transform to rate 
    output = Float64[]
    for pair_res in res
        push!(output, pair_res[1]/sum(pair_res))
    end
    return output
end

function count_false_pos(vecs::Vector{Vector{Int64}}, thr::Real = 286)
    res = [[0,0] for _ in 1:length(vecs[1])]

    for i in 1:length(vecs)
        for j in 1:length(vecs[i])
            if vecs[i][j] < thr && vecs[i][j] > 0 
                res[j][1] += 1
            else
                res[j][2] += 1
            end
        end
    end

    # transform to rate 
    output = Float64[]
    for pair_res in res
        push!(output, pair_res[1]/sum(pair_res))
    end
    return output
end

count_false_pos (generic function with 2 methods)

# Actual analysis

In [19]:
a = [] # mutations
for seed in 1:50
    Random.seed!(seed)
    push!(a, gma_tester(mut_range=0:0.005:1))
end

ErrorException: type String has no field aln